In [3]:
import pandas as pd
test_df = pd.read_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/建阳站多模型验证期结果.csv",
                      parse_dates=["Date"])
test_df_copy = test_df
test_df_copy = test_df_copy.iloc[:, :2]
test_df.reset_index(drop=True)
test_df = test_df.set_index('Date')
"""
读取训练数据
"""
train_df = pd.read_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/建阳站多模型率定期结果.csv",
                       parse_dates=["Date"])
train_df_copy = train_df
train_df_copy = train_df_copy.iloc[:, :2]
train_df.reset_index(drop=True)
train_df = train_df.set_index('Date')
test_df

,Qobs,XAJ,TVGM,TANK,GR4J,BP,SVR,LSTM,RNN
Date,,,,,,,,,
2013-03-27,357.285417,9.193883,0.000000,0.000000,1042.77000,440.787260,303.527337,470.890900,454.772770
2013-03-28,266.375000,12.697948,0.934126,4.533406,551.14210,226.363280,296.677261,246.736770,271.245000
2013-03-29,292.975000,36.728462,4.019340,5.295627,388.68440,209.675610,232.392399,249.087840,256.492130
2013-03-30,279.295833,80.326466,7.078621,89.600720,344.14320,299.446600,268.555454,331.467470,308.821300
2013-03-31,263.750000,60.581075,8.928750,90.898580,297.63830,239.122000,257.178080,274.810900,243.782490
...,...,...,...,...,...,...,...,...,...
2018-10-26,39.116250,62.387111,7.200014,52.107530,13.77444,57.210323,54.105041,48.253920,44.966537
2018-10-27,46.711250,88.995273,8.572588,58.079940,22.41679,122.920510,52.522373,105.119110,80.538630
2018-10-28,70.970417,83.566911,9.881388,79.087470,35.91110,110.055145,60.653979,99.763810,73.898160


In [4]:
# 导入进行线性回归的包

from sklearn.preprocessing import MinMaxScaler
# 导入随机森林包
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
from tool import Indicator
"""
案例一：XAJ+TANK
"""
# 处理数据
trainX_xaj = train_df.iloc[:, 1]
trainX_tank = train_df.iloc[:, 3]
trainy1 = train_df.iloc[:, :1]
trainx1 = pd.merge(trainX_xaj, trainX_tank, on="Date", how="left")
train_data1=pd.merge(trainy1,trainx1,on="Date",how="left")
testX_xaj = test_df.iloc[:, 1]
testX_tank = test_df.iloc[:, 3]
testy1 = test_df.iloc[:, :1]
testx1 = pd.merge(testX_xaj, testX_tank, on="Date", how="left")
test_data1=pd.merge(testy1,testx1,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled1 = scaler.fit_transform(train_data1)
df_for_testing_scaled1 = scaler.transform(test_data1)

# 划分数据
trainX1 = df_for_training_scaled1[:, 1:]
trainY1 = df_for_training_scaled1[:, :1]
testX1 = df_for_testing_scaled1[:, 1:]
testY1 = df_for_testing_scaled1[:, :1]


# 随机森林回归器
param_grid = {
    'n_estimators': [50, 100, 200],  #决策树的个数
    'max_depth': [3, 5, 7],  #最大树深，树太深会造成过拟合
}
rf = RandomForestRegressor()
# grid_model = KerasRegressor(build_fn=rf, verbose=1, validation_data=(testX, testY))
# grid_search = GridSearchCV(estimator=grid_model, param_grid=param_grid,
#                            cv=5)
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
grid_search = grid_search.fit(trainX1, trainY1)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict1 = model1.predict(trainX1)
train_predict1 = pd.DataFrame(train_predict1)
train_predict1.columns = ["RF1"]

test_predict1 = model1.predict(testX1)
test_predict1 = pd.DataFrame(test_predict1)
test_predict1.columns = ["RF1"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict1,len(test_data1.columns), axis=-1)
test_pred1 = scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict1),len(test_data1.columns))))[:, 0]
test_pred_copy1 = test_pred1
test_pred1 = pd.DataFrame(test_pred1)
test_pred1.columns = ["RF1"]

prediction_copies_array = np.repeat(train_predict1,len(test_data1.columns), axis=-1)
train_pred1 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict1),len(test_data1.columns))))[:, 0]
train_pred_copy1 = train_pred1
train_pred1=pd.DataFrame(train_pred1)
train_pred1.columns=['RF1']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred1['RF1'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred1['RF1'], test_df_copy['Qobs'])

/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vec

最佳参数组合：{'max_depth': 5, 'n_estimators': 200}
*******************率定期*******************
MAE:42.92460460534364
RMSE:97.52511193812681
NSE:0.859760860883085
RE:-0.304618523549083
Qmaxe:0.22686591149446697
Qmine:2.8413521286895134
*******************验证期*******************
MAE:56.51820841904418
RMSE:141.7434957339743
NSE:0.7499578555313454
RE:-8.181759028974179
Qmaxe:0.23605761816340765
Qmine:0.7024204812658823


(56.51820841904418,
 141.7434957339743,
 0.7499578555313454,
 -8.181759028974179,
 0.23605761816340765,
 0.7024204812658823)

In [5]:
"""
案例二：BP+RNN
"""
# 处理数据
trainX_BP = train_df.iloc[:, 5]
trainX_RNN = train_df.iloc[:, 7]
trainy2 = train_df.iloc[:, :1]
trainx2 = pd.merge(trainX_BP, trainX_RNN, on="Date", how="left")
testX_BP = test_df.iloc[:, 5]
testX_RNN = test_df.iloc[:, 7]
testy2 = test_df.iloc[:, :1]
testx2 = pd.merge(testX_BP, testX_RNN, on="Date", how="left")
train_data2=pd.merge(trainy2,trainx2,on="Date",how="left")
test_data2=pd.merge(testy2,testx2,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled2 = scaler.fit_transform(train_data2)
df_for_testing_scaled2 = scaler.transform(test_data2)

# 划分数据
trainX2 = df_for_training_scaled2[:, 1:]
trainY2 = df_for_training_scaled2[:, :1]
testX2 = df_for_testing_scaled2[:, 1:]
testY2 = df_for_testing_scaled2[:, :1]


# 随机森林回归器
param_grid = {
    'n_estimators': [50, 100, 200],  #决策树的个数
    'max_depth': [3, 5, 7],  #最大树深，树太深会造成过拟合
}
rf = RandomForestRegressor()
# grid_model = KerasRegressor(build_fn=rf, verbose=1, validation_data=(testX, testY))
# grid_search = GridSearchCV(estimator=grid_model, param_grid=param_grid,
#                            cv=5)
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
grid_search = grid_search.fit(trainX2, trainY2)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict2 = model1.predict(trainX2)
train_predict2 = pd.DataFrame(train_predict2)
train_predict2.columns = ["RF2"]

test_predict2 = model1.predict(testX2)
test_predict2 = pd.DataFrame(test_predict2)
test_predict2.columns = ["RF2"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict2,len(test_data2.columns), axis=-1)
test_pred2 = scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict2),len(test_data2.columns))))[:, 0]
test_pred_copy2 = test_pred2
test_pred2 = pd.DataFrame(test_pred2)
test_pred2.columns = ["RF2"]

prediction_copies_array = np.repeat(train_predict2,len(test_data2.columns), axis=-1)
train_pred2 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict2),len(test_data2.columns))))[:, 0]
train_pred_copy2 = train_pred2
train_pred2=pd.DataFrame(train_pred2)
train_pred2.columns=['RF2']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred2['RF2'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred2['RF2'], test_df_copy['Qobs'])

/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vec

最佳参数组合：{'max_depth': 5, 'n_estimators': 50}
*******************率定期*******************
MAE:37.52211103882104
RMSE:94.29309358871502
NSE:0.8689019925349989
RE:-0.38365574710035444
Qmaxe:0.19798185923814843
Qmine:2.3873445782590688
*******************验证期*******************
MAE:51.69426403733872
RMSE:139.46423309286396
NSE:0.7579346529244688
RE:-5.332902327740811
Qmaxe:0.2756150921179355
Qmine:0.5012122278674045


(51.69426403733872,
 139.46423309286396,
 0.7579346529244688,
 -5.332902327740811,
 0.2756150921179355,
 0.5012122278674045)

In [6]:
"""
案例三：BP+RNN+XAJ+TANK
"""
# 处理数据
# 处理数据
trainy3 = train_df.iloc[:, :1]
trainx3 = pd.merge(trainx1, trainx2, on="Date", how="left")
testy3 = test_df.iloc[:, :1]
testx3 = pd.merge(testx1, testx2, on="Date", how="left")
train_data3=pd.merge(trainy3,trainx3,on="Date",how="left")
test_data3=pd.merge(testy3,testx3,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled3 = scaler.fit_transform(train_data3)
df_for_testing_scaled3 = scaler.transform(test_data3)

# 划分数据
trainX3 = df_for_training_scaled3[:, 1:]
trainY3 = df_for_training_scaled3[:, :1]
testX3 = df_for_testing_scaled3[:, 1:]
testY3 = df_for_testing_scaled3[:, :1]


# 随机森林回归器
param_grid = {
    'n_estimators': [50, 100, 200],  #决策树的个数
    'max_depth': [3, 5, 7],  #最大树深，树太深会造成过拟合
}
rf = RandomForestRegressor()
# grid_model = KerasRegressor(build_fn=rf, verbose=1, validation_data=(testX, testY))
# grid_search = GridSearchCV(estimator=grid_model, param_grid=param_grid,
#                            cv=5)
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
grid_search = grid_search.fit(trainX3, trainY3)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict3 = model1.predict(trainX3)
train_predict3 = pd.DataFrame(train_predict3)
train_predict3.columns = ["RF3"]

test_predict3 = model1.predict(testX3)
test_predict3 = pd.DataFrame(test_predict3)
test_predict3.columns = ["RF3"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict3,len(test_data3.columns), axis=-1)
test_pred3= scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict3),len(test_data3.columns))))[:, 0]
test_pred_copy3 = test_pred3
test_pred3 = pd.DataFrame(test_pred3)
test_pred3.columns = ["RF3"]

prediction_copies_array = np.repeat(train_predict3,len(test_data3.columns), axis=-1)
train_pred3 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict3),len(test_data3.columns))))[:, 0]
train_pred_copy3 = train_pred3
train_pred3=pd.DataFrame(train_pred3)
train_pred3.columns=['RF3']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred3['RF3'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred3['RF3'], test_df_copy['Qobs'])

/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vec

最佳参数组合：{'max_depth': 5, 'n_estimators': 50}
*******************率定期*******************
MAE:36.501941583799415
RMSE:90.1889839859957
NSE:0.8800657258524345
RE:-0.2867211450542209
Qmaxe:0.21711847764245157
Qmine:2.306815371665799
*******************验证期*******************
MAE:51.119936812707444
RMSE:137.2862687648564
NSE:0.7654361184165919
RE:-5.140822723220375
Qmaxe:0.2677697990004921
Qmine:0.46552308351102795


(51.119936812707444,
 137.2862687648564,
 0.7654361184165919,
 -5.140822723220375,
 0.2677697990004921,
 0.46552308351102795)

In [7]:
test_df_full = pd.concat([test_df_copy, test_pred1, test_pred2, test_pred3], axis=1)
test_df_full = test_df_full.reset_index(drop=True)
test_df_full = test_df_full.set_index('Date')
test_df_full.to_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/JY_RF1Test.csv")
train_df_full = pd.concat([train_df_copy, train_pred1, train_pred2, train_pred3], axis=1)
train_df_full = train_df_full.reset_index(drop=True)
train_df_full = train_df_full.set_index('Date')
train_df_full.to_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/JY_RF1Train.csv")
test_df_full

,Qobs,RF1,RF2,RF3
Date,,,,
2013-03-27,357.285417,40.475047,543.725656,580.878665
2013-03-28,266.375000,40.475047,245.061752,205.512852
2013-03-29,292.975000,40.475047,243.560702,208.501908
2013-03-30,279.295833,84.840290,341.472999,558.081932
2013-03-31,263.750000,81.532139,244.620659,214.800334
...,...,...,...,...
2018-10-26,39.116250,58.010707,47.523517,46.300462
2018-10-27,46.711250,84.710849,103.072130,100.631856
2018-10-28,70.970417,88.207326,102.662310,99.466474
